# Importing Libraries

In [98]:
import pandas as pd
import numpy as np
import os

# Importing Data Frame

In [99]:
# Creating the path for the data
path = r'C:\Users\sdaon\Documents\Instacart Basket Analysis'

In [100]:
# Importing orders_products_merged
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_hourly_daily.pkl'))

In [101]:
df.shape

(1000000, 15)

In [102]:
df.head(25)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,195,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,195,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,195,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,195,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,195,Soda,77,7,9.0,both
5,3367565,1,6,2,7,19.0,196,1,1,195,Soda,77,7,9.0,both
6,550135,1,7,1,9,20.0,196,1,1,195,Soda,77,7,9.0,both
7,3108588,1,8,1,14,14.0,196,2,1,195,Soda,77,7,9.0,both
8,2295261,1,9,1,16,0.0,196,4,1,195,Soda,77,7,9.0,both
9,2550362,1,10,4,8,30.0,196,1,1,195,Soda,77,7,9.0,both


In [103]:
# step 2 - Grouping data by department_id and aggregate by order_number mean
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


Step 3: While the means are close, department_id 17 hasthe lowest number.

In [104]:
# Step 4- creating a loyalty flag for existing customers using the transform() and loc()
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [105]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [106]:
# Checking the loyalty numbers
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15891077
Loyal customer      10293737
New customer         6249398
Name: loyalty_flag, dtype: int64

In [107]:
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(20)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


In [108]:
# step 5 - Checking the basic statistics of the product prices for each loyalty category
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max', 'sum']})

prices                           
                       mean  min      max          sum
loyalty_flag                                          
Loyal customer    10.388747  1.0  99999.0  106939028.9
New customer      13.294370  1.0  99999.0   83081810.0
Regular customer  12.496203  1.0  99999.0  198578123.1

Depending on the results, Loyar customers buy cheaper products. I can see that, regular customers paid more amounth of money already but we already know that there are more regular customers than loyal customers.

In [109]:
# Step 6 - creating an avg_price column and populating data by user_id and prices
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [110]:
# Creating a spending flag for each user based on the average price
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_loc'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_loc'] = 'High spender'

In [111]:
# Checking the numbers of spenders
ords_prods_merge['spending_loc'].value_counts(dropna = False)

Low spender     31798751
High spender      635461
Name: spending_loc, dtype: int64

In [112]:
ords_prods_merge.head(25)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,...,prices,_merge,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_loc
0,2539329,1,1,2,8,NaN,196,1,0,195,...,9.0,both,Mid-range product,Regularly busy,Regular days,Average orders,10,New customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,195,...,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,195,...,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,195,...,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,195,...,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender
5,3367565,1,6,2,7,19.0,196,1,1,195,...,9.0,both,Mid-range product,Regularly busy,Regular days,Average orders,10,New customer,6.367797,Low spender
6,550135,1,7,1,9,20.0,196,1,1,195,...,9.0,both,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender
7,3108588,1,8,1,14,14.0,196,2,1,195,...,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender
8,2295261,1,9,1,16,0.0,196,4,1,195,...,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender
9,2550362,1,10,4,8,30.0,196,1,1,195,...,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender


In [113]:
# Step 7 - Creating a frequent_median column to populate frequency data based on days_since_prior_order
ords_prods_merge['frequent_median'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [114]:
# Creating frequent_flag and populating data based on frequent_median
ords_prods_merge.loc[ords_prods_merge['frequent_median'] > 20, 'frequent_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['frequent_median'] > 10) & (ords_prods_merge['frequent_median'] <= 20), 'frequent_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['frequent_median'] <= 10, 'frequent_flag'] = 'Frequent customer'

In [115]:
# Checking the numbers of customers based on frequent_flag
ords_prods_merge['frequent_flag'].value_counts(dropna = False)

Frequent customer        21577409
Regular customer          7217134
Non-frequent customer     3639669
Name: frequent_flag, dtype: int64

In [116]:
ords_prods_merge.head(25)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,...,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_loc,frequent_median,frequent_flag
0,2539329,1,1,2,8,NaN,196,1,0,195,...,Mid-range product,Regularly busy,Regular days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,195,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,195,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,195,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,195,...,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
5,3367565,1,6,2,7,19.0,196,1,1,195,...,Mid-range product,Regularly busy,Regular days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
6,550135,1,7,1,9,20.0,196,1,1,195,...,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
7,3108588,1,8,1,14,14.0,196,2,1,195,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
8,2295261,1,9,1,16,0.0,196,4,1,195,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
9,2550362,1,10,4,8,30.0,196,1,1,195,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [117]:
# Step 9 - Export ords_prods_merge dataframe as a pickle file
ords_prods_merge.to_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_hourly_daily_aggregated.pkl'))